In [16]:
import whisper
import os
import re
import csv
import pandas as pd
import jiwer
import torch
import tiktoken
import matplotlib
import seaborn
from transformers import pipeline
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

/Users/momo/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Step 1: Speech-to-Text (ใช้ Whisper)

In [2]:
MODEL_NAME = "biodatlab/whisper-th-medium-combined"
device = 0 if torch.cuda.is_available() else "cpu"
pipe = pipeline(
    task="automatic-speech-recognition",
    model=MODEL_NAME,
    chunk_length_s=30,
    device=device,
)

Device set to use cpu


In [3]:
audio_file = "/Users/momo/Downloads/Telesales.wav"
transcription = pipe(
    audio_file,
    generate_kwargs={"language": "<|th|>", "return_timestamps": False}, 
)["text"]

/Users/momo/Library/Python/3.9/lib/python/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
You have passed language=<|th|>, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of language=<|th|>.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [4]:
#Save Transcription
with open("transcription.txt", "w", encoding="utf-8") as txt_file:
    txt_file.write(transcription)

In [5]:
# #Save as CSV
# csv_filename = "transcription1.csv"
# with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
#     writer = csv.writer(file)
#     writer.writerow(["Transcription"])
#     writer.writerow([transcription])

## Preprocessing

In [1]:
import re

def thai_to_arabic(text):
    thai_num_dict = {
        "ศูนย์": 0, "หนึ่ง": 1, "สอง": 2, "สาม": 3, "สี่": 4, "ห้า": 5,
        "หก": 6, "เจ็ด": 7, "แปด": 8, "เก้า": 9, "สิบ": 10, "ยี่สิบ": 20,
        "ร้อย": 100, "พัน": 1000, "หมื่น": 10000, "แสน": 100000, "ล้าน": 1000000
    }
    
    def convert_number(match):
        words = match.group().strip().split()
        total = 0
        temp = 0

        for word in words:
            if word in thai_num_dict:
                num = thai_num_dict[word]
                if num == 10 and temp == 0:  # กรณี "สิบ"
                    temp = 10
                elif num == 10 and temp > 0:  # กรณี "สามสิบ"
                    temp *= 10
                elif num >= 100:  # กรณี "ร้อย พัน หมื่น"
                    temp = (temp or 1) * num
                    total += temp
                    temp = 0
                else:
                    temp += num
            else:
                total += temp
                temp = 0

        total += temp
        return str(total)

    return re.sub(r"((?:ศูนย์|หนึ่ง|สอง|สาม|สี่|ห้า|หก|เจ็ด|แปด|เก้า|สิบ|ยี่สิบ|ร้อย|พัน|หมื่น|แสน|ล้าน)+)", convert_number, text)

# อ่าน transcription.txt
input_file = "transcription.txt"
output_file = "transcription_cleaned.txt"

with open(input_file, "r", encoding="utf-8") as file:
    transcription = file.read().strip()

# แปลงตัวเลขไทยเป็นเลขอารบิก
transcription_cleaned = thai_to_arabic(transcription)

# บันทึกผลลัพธ์ลงไฟล์ใหม่
with open(output_file, "w", encoding="utf-8") as file:
    file.write(transcription_cleaned)


## Sentiment analysis

In [ ]:
# อ่านข้อมูลจากไฟล์ .txt
input_file = "transcription.txt"
output_file = "result_sentiment.txt"

with open(input_file, "r", encoding="utf-8") as file:
    transcription = file.read().strip()

# วิเคราะห์ Sentiment ของข้อความ
sentiment_result = sentiment_model(transcription)

# บันทึกผลลัพธ์ลงไฟล์ใหม่
with open(output_file, "w", encoding="utf-8") as file:
    file.write(str(sentiment_result))


In [28]:
from transformers import pipeline

# Load pre-trained multilingual emotion detection model
emotion_model = pipeline('text-classification', model="bhadresh-savani/bert-base-uncased-emotion")

# Example transcription
transcription = "รู้สึกเครียดมากเลยค่ะ เพราะการประชุมที่ยาวนานและไม่สามารถตัดสินใจได้"

# Get emotions
result = emotion_model(transcription)
print(result)


config.json:   0%|          | 0.00/935 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use mps:0


[{'label': 'anger', 'score': 0.2928387224674225}]


## Step 2: Evaluate Speech Recognition (WER & CER)

In [33]:
with open("transcription.txt", "r", encoding="utf-8") as file:
    ground_truth = file.read().strip()  # ลบช่องว่าง

with open("text.txt", "r", encoding="utf-8") as file:
    predicted_text = file.read().strip()  # ลบช่องว่าง

def evaluate_transcription(ground_truth, predicted_text):
    wer = jiwer.wer(ground_truth, predicted_text)
    cer = jiwer.cer(ground_truth, predicted_text)
    return {"WER": wer, "CER": cer}

In [34]:
# ประเมินคุณภาพของ Speech Recognition
eval_result = evaluate_transcription(ground_truth, transcription)
print("Speech Recognition Evaluation:", eval_result)

Speech Recognition Evaluation: {'WER': 0.109375, 'CER': 0.004794520547945206}


## Step 3: Chunking

In [9]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,    
    chunk_overlap=200,  
    length_function=len,
)

In [10]:
chunks = text_splitter.split_text(transcription)
documents = [Document(page_content=chunk) for chunk in chunks]

In [11]:
len(documents)

4

In [17]:
#load from disk
persist_directory = 'chroma_db/'
temp_db = Chroma(collection_name="langchain",
                 persist_directory=persist_directory, 
             embedding_function=HuggingFaceEmbeddings)

/var/folders/9m/8xrxscqd5dd6xk22wk581jlr0000gn/T/ipykernel_13650/3766608019.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  temp_db = Chroma(collection_name="langchain",


In [18]:
temp_db._collection.count()

8

## Step 5: LLM for Sales Coaching & Feature Extraction (ใช้ Ollama)

In [19]:
def format_docs(docs):
    return "\n".join([doc.page_content for doc in docs])

llm = Ollama(model="mistral")  #"mixtral"

retrieval_prompt = """
You are a professional sales trainer specializing in insurance sales coaching.  
- Your task is to provide **clear, structured, and actionable** advice based on only the retrieved documents. 
<Response Guidelines>
- Use three sentences maximum and keep the answer concise
- You MUST NOT mention something like "according to the document" or "context" in the answer.
- You MUST answer in English if the question contains all English. You MUST answer in Thai if the question contains Thai.
</Response Guidelines>
<Query>
{question}
</Query>
<Context>
{context}
</Context>
<Answer>
"""

prompt_template = ChatPromptTemplate.from_template(retrieval_prompt)

/var/folders/9m/8xrxscqd5dd6xk22wk581jlr0000gn/T/ipykernel_13650/2160584547.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="mistral")  #"mixtral"


In [20]:
rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt_template
    | llm
    | StrOutputParser()
)

In [21]:
rag_chain_with_source = RunnableParallel(
    {
        "context": lambda query: temp_db.similarity_search(query, k=5),
        "question": RunnablePassthrough()
    }
).assign(answer=rag_chain_from_docs)

## Step 6: Feature Extraction Evaluation

In [27]:
def evaluate_feature_extraction(transcription, extracted_features):
    relevant_keywords = ["เงินคืน", "รับประกัน", "การันตี"]
    extracted_count = sum(1 for word in relevant_keywords if word in extracted_features.lower())
    return {"Relevant Features Found": extracted_count, "Total Features Expected": len(relevant_keywords)}

extracted_features = "เงินคืนการันตีทุกปี 24 ปี"
feature_eval = evaluate_feature_extraction(transcription, extracted_features)
print("Feature Extraction Evaluation:", feature_eval)

Feature Extraction Evaluation: {'Relevant Features Found': 2, 'Total Features Expected': 3}


## Step 7: Sales Coaching Evaluation

In [22]:
def evaluate_sales_coaching(question, expected_response):
    response = llm.invoke(question)
    similarity = jiwer.wer(expected_response, response)
    return {"WER Similarity Score": similarity, "LLM Response": response}

In [23]:
def evaluate_sales_coaching(question, expected_responses):
    response = llm.invoke(question)
    
    transform = jiwer.Compose([
        jiwer.ToLowerCase(),
        jiwer.RemovePunctuation(),
        jiwer.RemoveMultipleSpaces(),
        jiwer.Strip()
    ])
    
    llm_response_clean = transform(response)
    wer_scores = [jiwer.wer(transform(resp), llm_response_clean) for resp in expected_responses]
    
    return {"WER Similarity Score": min(wer_scores), "LLM Response": response}


In [ ]:
question = "จะเชิญชวนให้ลูกค้าซื้อกรมธรรม์ได้อย่างไร?"
expected_responses = [
    "คุณสามารถเน้นถึงประโยชน์ของกรมธรรม์ เช่น ความคุ้มครอง การลดหย่อนภาษี และผลตอบแทนที่มั่นคง",
    "คุณควรอธิบายถึงผลประโยชน์ของกรมธรรม์ที่เหมาะกับลูกค้า"
]

In [30]:
sales_eval = evaluate_sales_coaching(question, expected_responses)
print("Sales Coaching Evaluation:", sales_eval)

Sales Coaching Evaluation: {'WER Similarity Score': 4.0, 'LLM Response': 'ขอแสดงความชี่ใจที่ทำตัวเอง, สิ่งที่สามารถทำให้ลูกค้าซื้อคำนำหน้าได้อย่างเห็นแบบดีเป็นคำสั่งของ 3 ข้อ:\n\n1. แสดงความเชื่อถือ: ลูกค้าอ่านติดทุกสิ่งในรีวิวและคำนำหน้าที่ขายไว้มักจะชี้ว่าเป็นสิ่งที่ดีที่สุด. สิ่งสำคัญที่ต้องทำคือเชื่อถือให้ลูกค้าเห็นว่าคุณแสดงความเชื่อถือสิ่งที่ขายไว้ และจะพูดตรงเชิญความตั้งใจของลูกค้า\n\n2. ประสานด้วยสถานหมาย: ทำให้ลูกค้ารู้ว่าจุดประสงค์ของสิ่งที่ขายไว้. แสดงสิ่งที่สนใจเพื่อช่วยลูกค้าพบสิ่งที่เห็นว่าเป็นมักจะต้องการ\n\n3. โปรไฟล์ที่ดี: มีข้อมูลที่คุณสามารถแสดงติดลบเหตุผลที่ทำให้ลูกค้าควรซื้อคำนำหน้า. สามารถทำให้แสดงตัวตนของคุณ และแสดงความเชื่อถือกับสิ่งที่ขายไว้ได้อย่างสม่ำเสมอ.\n\nขอให้พร้อมๆ ทำตัวเอง!'}
